In [1]:
from train import *

%load_ext autoreload
%autoreload 2

/Users/nmm/miniforge3/envs/env_gdl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os
sys.path.append("../")

from synthetic.net import load_net

# 1. Make dataset out of the folder containing generated nets

In [3]:
PATH_TO_FOLDER = "../space_seg/data/data_v2/v100_e552/"

## 1.a) Get files path from folder

In [4]:
files_paths = []
for path, subdirs, files in os.walk(PATH_TO_FOLDER):
    for name in files:
        files_paths += [os.path.join(path, name)]
        
files_paths = [f for f in files_paths
               if f.endswith(".pickle")
              ]

## 1.b) Load nets from paths

In [5]:
nets = [load_net(f, directed=False) for f in files_paths]

## 1.c) Convert nets to `dgl`

In [6]:
import dgl
from pyintergraph import igraph2nx

In [7]:
def igraph2dgl(ig):
    return dgl.from_networkx(igraph2nx(ig))

In [8]:
dgl_graphs = [igraph2dgl(net.graph) for net in nets]

# 2. Loading & adapting config and models to be fine-tuned

In [9]:
PATH_TO_MODEL = "./saved/models/PGCL_default/"
RUN_ID = "0227_102126"

In [10]:
config = ConfigParser.from_json(
    json_path = PATH_TO_MODEL+RUN_ID+"/config.json"
    , resume = PATH_TO_MODEL+RUN_ID+"/model_best.pth"
)

In [11]:
config._config["do_train"] = True


fine_tuning_epochs = 4

config._config["trainer"]["epochs"] = config._config["trainer"]["epochs"] + fine_tuning_epochs

## 2.a) Change dataset

In [12]:
config._config["dataset"]["args"]["dgl_graphs"] = dgl_graphs

In [13]:
dataset = config.init_obj('dataset', module_dataset)

In [14]:
print(dataset)

┌-----------------------------------------------------------┐
|                        PGCLDataset                        |
├-----------------------------------------------------------┤
|number of graphs                       |                704|
|nodes — tot                            |              70400|
|nodes — mean                           |              100.0|
|nodes — median                         |              100.0|
|nodes — min                            |                100|
|nodes — max                            |                100|
|edges — tot                            |             777216|
|edges — mean                           |             1104.0|
|edges — median                         |             1104.0|
|edges — min                            |               1104|
|edges — max                            |               1104|
└-----------------------------------------------------------┘



## 2.b) Load model

In [15]:
main(config)

🌐 Loading Dataset...
Dataset loaded:
┌-----------------------------------------------------------┐
|                        PGCLDataset                        |
├-----------------------------------------------------------┤
|number of graphs                       |                704|
|nodes — tot                            |              70400|
|nodes — mean                           |              100.0|
|nodes — median                         |              100.0|
|nodes — min                            |                100|
|nodes — max                            |                100|
|edges — tot                            |             777216|
|edges — mean                           |             1104.0|
|edges — median                         |             1104.0|
|edges — min                            |               1104|
|edges — max                            |               1104|
└-----------------------------------------------------------┘

🤖 Model instantiated:PGCL (pgcl)

/Users/nmm/Desktop/CMB_CSS_PDM/main_project/DL_module/Models/model_util.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=int)


Train Epoch: 18 [0/6 (0%)] Loss: 23.735561
    epoch          : 18
    loss           : 23.318254470825195
    time           : 00:00:08
    no improvement : 1
Train Epoch: 19 [0/6 (0%)] Loss: 23.479342
    epoch          : 19
    loss           : 23.274819056193035
    time           : 00:00:06
    no improvement : 2
Train Epoch: 20 [0/6 (0%)] Loss: 23.485237
    epoch          : 20
    loss           : 23.22943051656087
    time           : 00:00:06
    no improvement : 3
Train Epoch: 21 [0/6 (0%)] Loss: 23.411306
    epoch          : 21
    loss           : 23.229420979817707
    time           : 00:00:08
    no improvement : 4
Train Epoch: 22 [0/6 (0%)] Loss: 23.484447
    epoch          : 22
    loss           : 23.19984022776286
    time           : 00:00:06
    no improvement : 0
Saving current best: model_best.pth ...
Train Epoch: 23 [0/6 (0%)] Loss: 23.333153
    epoch          : 23
    loss           : 23.11913013458252
    time           : 00:00:06
    no improvement : 0
Sav